In [1]:
import os
import numpy as np
from scipy.stats import mode
import matplotlib.pyplot as plt
from lib_split import loadmat

ROOT = 'C:/Users/amcca/switchdrive/PhD/RR/'
OUTCOME_TYPE = 'procedural'
OUTCOME_GROUP = 'LT'

### Import data
Import data from matlab and put into dictionary/matrix

In [2]:
tau = 1
m = 2
coef = 0.1

# load in matlab data with appropriate params
filename = 'tau' + str(tau) + '_dim' + str(m) + '_coef' + str(coef) + '.mat'
bl_mat = loadmat(ROOT + 'data_split/rr_indices/baseline/' + filename)
end_mat = loadmat(ROOT + 'data_split/rr_indices/end ablation/' + filename)

# get each AF complexity index in column, each row corresponds to one patient/file combo only
data_dict_bl = bl_mat['rr_indices_struct']
feat_names = data_dict_bl.keys()
data_matrix_bl = np.stack(list(data_dict_bl.values())).transpose()
Xbl = data_matrix_bl

data_dict_end = end_mat['rr_indices_struct']
data_matrix_end = np.stack(list(data_dict_end.values())).transpose()
Xend = data_matrix_end

# for clinical outcomes, SR=1, AR=0
y_clin = np.array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# for procedural outcomes, LT=1, NT=0
y_proc=np.array([1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1])

### account for multiples files per patient
There are several RR segments and associated metrics/outcomes per patient

In [3]:
pts = np.arange(1,42)
pts = np.delete(pts, 32)
n_pts = len(pts)
n_files_per_pt = 5
pt_file_idx = 0

if OUTCOME_TYPE == 'clinical':
    y_tmp = y_clin
elif OUTCOME_TYPE == 'procedural':
    y_tmp = y_proc

# several RR segments for each patient
group = np.zeros((n_pts*n_files_per_pt))
y = np.zeros((n_pts*n_files_per_pt))

# replicate outcome and group n_file_per_pt number of times
for pt_idx in range(n_pts):
    for file_nb in range(n_files_per_pt):
        y[pt_file_idx] = y_tmp[pt_idx]
        group[pt_file_idx] = pts[pt_idx]
        pt_file_idx += 1
        
# extract indices of data matrix to keep, based on outcome type and group desired
if OUTCOME_TYPE == 'clinical':
    if OUTCOME_GROUP == 'SR':
        keep_idx = np.nonzero(y==1)[0]
    elif OUTCOME_GROUP == 'AR':
        keep_idx = np.nonzero(y==0)[0]
elif OUTCOME_TYPE == 'procedural':
    if OUTCOME_GROUP == 'LT':
        keep_idx = np.nonzero(y==1)[0]
    elif OUTCOME_GROUP == 'NT':
        keep_idx = np.nonzero(y==0)[0]
        
Xbl = Xbl[keep_idx,:]
Xend = Xend[keep_idx,:]
group = group[keep_idx]

### Statistical significance: baseline to end ablation
The purpose of this notebook is to find which RR interval indices (features) have sig different values prior to ablation (baseline) and at the end of ablation, within the same patient group. Find which features are most discriminative across cross-validated folds, using one-way ANOVA or rank-sums for statistically sig differences in means between groups.

In [4]:
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_selection import SelectKBest, f_classif
from statsmodels.stats.diagnostic import lilliefors
from scipy.stats import ranksums

In [5]:
n_folds = 3
gss = GroupShuffleSplit(n_splits=n_folds, test_size=0.2, random_state=0)
fnorm = []
pnorm = []

plillie_bl = np.zeros((n_folds, len(feat_names)))
plillie_end = np.zeros((n_folds, len(feat_names)))
prank = np.zeros((n_folds, len(feat_names)))


for i, (train, test) in enumerate(gss.split(Xbl, groups=group)):
        # enumerate over data folds and features
        for k, feat_name in enumerate(feat_names):
            # use lilliefors statistic to determine whether data dist normal. If returned
            # p-value is lower than some threshold (e.g. 0.05), then reject null hypothesis
            # that data are normally distributed
            ksstat_x, plillie_bl[i,k] = lilliefors(np.concatenate((Xbl[train,k],Xend[train,k])), dist='norm')
            ksstat_x, plillie_end[i,k] = lilliefors(Xend[train,k], dist='norm')
            
            # calculate ranksum statistic
            statistic, prank[i,k] = ranksums(Xbl[train,k], Xend[train,k])
            
        X = np.vstack([Xbl[train,:], Xend[train,:]])
        y_tmp = np.concatenate([np.zeros(len(train)), np.ones(len(train))])
        # calculate anova p-values for all features in fold, and append
        f_fold, p_fold = f_classif(X, y_tmp)
        fnorm.append(f_fold)
        pnorm.append(p_fold)
        
# calculate mean lilliefors/p-values across folds
fnorm = np.vstack(fnorm)
pnorm = np.vstack(pnorm)
            
mean_f = np.mean(fnorm, axis=0)
mean_plillie_bl = np.mean(plillie_bl, axis=0)
mean_plillie_end = np.mean(plillie_end, axis=0)
mean_pnorm = np.mean(pnorm, axis=0)
mean_prank = np.mean(prank, axis=0)

# calculate mean and std of baseline and end ablation data
mean_Xbl = np.mean(Xbl, axis=0)
mean_Xend = np.mean(Xend, axis=0)
std_Xbl = np.std(Xbl, axis=0)
std_Xend = np.std(Xend, axis=0)

## Feature statistical significance
Inspect mean/std of features values in each group, along with p-value (either rank-sum or ANOVA).

In [6]:
from prettytable import PrettyTable

# extract indices of data matrix to keep, based on outcome type and group desired
if OUTCOME_TYPE == 'clinical':
    if OUTCOME_GROUP == 'SR':
        title_string = 'Clinical outcomes: SR group'
    elif OUTCOME_GROUP == 'AR':
        title_string = 'Clinical outcomes: AR group'
elif OUTCOME_TYPE == 'procedural':
    if OUTCOME_GROUP == 'LT':
        title_string = 'Procedural outcomes: LT group'
    elif OUTCOME_GROUP == 'NT':
        title_string = 'Procedural outcomes: NT group'

# table with mean values
t = PrettyTable(['Feature', 'baseline', 'end ablation', 'p-anova', 'p-rank', 'plillie'])
for i, feat_name in enumerate(feat_names):
    t.add_row([feat_name, mean_Xbl[i], mean_Xend[i], mean_pnorm[i], mean_prank[i], mean_plillie_bl[i]])

t.title = title_string
t.float_format = '0.4'
print(t)

# table with standard deviation values
t = PrettyTable(['Feature', 'std baseline', 'std end ablation', 'p-anova'])
for i, feat_name in enumerate(feat_names):
    t.add_row([feat_name, std_Xbl[i], std_Xend[i], mean_pnorm[i]])

t.title = title_string
t.float_format = '0.4'
print(t)

+----------------------------------------------------------------+
|                 Procedural outcomes: LT group                  |
+---------+----------+--------------+---------+--------+---------+
| Feature | baseline | end ablation | p-anova | p-rank | plillie |
+---------+----------+--------------+---------+--------+---------+
|   rec   |  0.0091  |    0.0218    |  0.0007 | 0.0084 |  0.0010 |
|   det   |  0.1660  |    0.2564    |  0.0001 | 0.0005 |  0.0016 |
|   div   |  0.5576  |    0.4128    |  0.0001 | 0.0000 |  0.0010 |
|  sampen |  4.1327  |    2.9825    |  0.0016 | 0.0015 |  0.0010 |
|  pnn20  |  0.4400  |    0.3824    |  0.0000 | 0.0000 |  0.1830 |
|  pnn50  |  0.3788  |    0.2937    |  0.0000 | 0.0000 |  0.0123 |
|   sdnn  | 170.6555 |   114.2952   |  0.0000 | 0.0000 |  0.0581 |
|  rmssd  | 242.1081 |   155.3529   |  0.0000 | 0.0000 |  0.0636 |
+---------+----------+--------------+---------+--------+---------+
+-----------------------------------------------------+
|     